# Quantum-Optimizated Training of Boolean Function Networks

## Introduction

This project is essentially an extension of the research I did with [Stephen Wolfram](https://en.wikipedia.org/wiki/Stephen_Wolfram) back in 2016. The goal is to construct a typical feed-forward neural network from a pseudo-mesh of boolean functions of two inputs. By a boolean function, I mean a function of the form:

$f : \{0, 1\}^{k} \rightarrow \{0, 1\}$ where $k \in \mathbb{Z}^{0+}$

For our purposes we are concerned with boolean functions of arity $k = 2$, each with the Python signature:

```python
def f(a: bool, b: bool) -> bool
```

There are 16 possible such functions, which are enumerated according to the following table:

| n | Definition | Name |
|---|------------|------|
| 0 | $0$ | FALSE |
| 1 | $a \land b$ | a AND b |
| 2 | $a \land \lnot b$ | a AND NOT b |
| 3 | $a$ | a |
| 4 | $\lnot a \land b$ | NOT a AND b |
| 5 | $b$ | b |
| 6 | $a \veebar b$ | a XOR b |
| 7 | $a \lor b$ | a OR b |
| 8 | $a \overline{\vee} b$ | a NOR b |
| 9 | $a \odot b$ | a XNOR b |
| 10 | $\lnot b$ | NOT b |
| 11 | $a \lor \lnot b$ | a OR NOT b |
| 12 | $\lnot a$ | NOT a |
| 13 | $\lnot a \lor b$ | NOT a OR b |
| 14 | $a \overline{\land} b$ | a NAND b |
| 15 | $1$ | TRUE |

To facilitate the creation of the $n$th boolean function of arity 2, in the accompanying library we define a `BooleanFunction` like so:

In [1]:
from qbn.bf import BooleanFunction

# Define f to be the 6th boolean function of two inputs (XOR).
f = BooleanFunction(6)

# Evaluate f(a, b).
f(True, True)

False

Consider a feed-forward "neural network" comprised of some combination of boolean functions as neurons. We concieve this network as being comprised of _layers_ represented by an array of boolean functions.

In [2]:
from qbn.network import Layer, LayerList

# The following will produce a layer consisting of the 3rd, 6th, and 14th boolean functions.
layer = Layer.build([3, 6, 14])

# We define layers as lists of boolean functions.
assert layer[2] == BooleanFunction(14)

# We can stack layers on top of each other to form a list.
layer_list = LayerList.build([[3, 5], [2, 8]])

# For instance, the first layer is made up of the 3rd and 5th boolean functions.
assert layer_list[0] == Layer.build([3, 5])

To "evaluate" this network on a particular input, we partition an input array of boolean values such that values are paired with each other and a corresponding boolean function of the first layer of the network. The outputs of the first layer forms an array which is then passed as input into the second layer of the network, and so on.

In [3]:
input_array = [True, False, True, True, False]

# Both Layer and LayerList implement __call__().
layer_list(input_array)

[False, False, True, True, False]

Such a network forms a [Boolean Circuit](https://en.wikipedia.org/wiki/Boolean_circuit) and is thus _P-complete_. In other words, there is no way to deduce or predict the output of the network other than by evaluating it. In addion, back-propagation via gradient descent on the network is impossible. We are thus confined to a Monte Carlo training mechanism in which we select and change a random neuron in the network to a new random boolean function.

### Classical Training Procedure

As stated above, we must unfortunately result to a pseudo-random training algorithm. At each step of the training process, we select and replace the boolean function of a neuron with a new random boolean function. We then determine the accuracy of the network by discerning the average proportion of a each class of inputs that produce the same output. While we _can_ make minor optimizations like preferring to tweak neurons that topologically map to areas of the input that are more active (like the centers of images in MNIST data), in general our training process is unnessessarily long.

To make things a little more friendly, we'll encapsulate our network into a new class called `Network` which contains the list of layers that comprise the "physical" network and a mapping of outputs to their corresponding classes. The latter is essentially a `dict[list[bool], str]` that might have entries like so:

```
{
  [True, False, True, True, False]: "Cat",
  [False, True, False, True, True]: "Dog"
}
```

Since `list[bool]` types aren't hashable, we introduce a wrapper class called `BooleanArray` such that our mapping is now of the form `dict[BooleanArray, str]`. We finally wrap this into a new class called `ClassificationMap`. This class serves a duel purpose:

1. Translating `LayerList` outputs to `str` classifications.
2. Representing training data inputs to their corresponding classifications.

#### Left vs. Right

Probably one of the simplest classification problems we could do with our network would be to classify whether `True` boolean values are located on the "left" side of an array, the "right" side of an array, or "both" sides (we omit the special "neither" case for demonstrative purposes). Let's consider a set of input arrays of length 8 - the training data for the network might look like the following:

In [4]:
from qbn.bf import BooleanArray
from qbn.training import ClassificationMap

lvr_training = ClassificationMap({
    BooleanArray([True, True, True, True, False, False, False, False]): "left",
    BooleanArray([True, True, False, True, False, False, False, False]): "left",
    BooleanArray([False, True, True, True, False, False, False, False]): "left",
    BooleanArray([False, True, False, False, False, False, False, False]): "left",
    BooleanArray([False, False, False, False, False, False, False, True]): "right",
    BooleanArray([False, False, False, False, False, False, True, True]): "right",
    BooleanArray([False, False, False, False, False, True, False, True]): "right",
    BooleanArray([False, False, False, False, False, False, True, False]): "right",
    BooleanArray([False, True, False, False, False, False, True, False]): "both",
    BooleanArray([True, False, False, False, False, False, True, True]): "both",
    BooleanArray([True, True, True, False, False, True, True, True]): "both",
    BooleanArray([True, False, False, True, False, False, True, False]): "both",
    BooleanArray([True, True, True, True, True, True, True, True]): "both",
})

lvr_training

{11110000 -> left, 11010000 -> left, 01110000 -> left, 01000000 -> left, 00000001 -> right, 00000011 -> right, 00000101 -> right, 00000010 -> right, 01000010 -> both, 10000011 -> both, 11100111 -> both, 10010010 -> both, 11111111 -> both}

Let's create a new network for processing the training data. We'll arbitrarily create an 8x4 network with a random set of boolean functions.

In [5]:
from qbn.network import Network

# Create a new, randomly initialized network of 4 layers of 8 neurons each.
lvr_net = Network.build([8, 8, 8, 8])

# Let's see what functions were randomly chosen.
lvr_net.layers

[4, 9, 9, 6, 9, 13, 9, 6]
[9, 6, 13, 13, 4, 14, 14, 14]
[2, 9, 11, 9, 1, 1, 4, 11]
[4, 13, 14, 7, 2, 6, 7, 14]

Without training the network, we can check its current accuracy against the test data. The validity of the network is determined by whether each class is assigned a unique output.

In [6]:
lvr_net.accuracy(lvr_training)

Valid?:            True
Overall Accuracy:  0.55
Accuracy By Class: {'left': 0.25, 'right': 1.0, 'both': 0.4}

Classifications:
{01100111 -> left, 10100111 -> right, 10100011 -> both}

Now let's train the network. As the training process runs, it will output the new accuracy score. The `goal` parameter specifies our intended target accuracy, defaulting to `0.95`.

In [7]:
lvr_net.train_classical(lvr_training, goal=0.9)

New Accuracy: 0.63
New Accuracy: 0.72


Valid?:            True
Overall Accuracy:  0.72
Accuracy By Class: {'left': 1.0, 'right': 0.75, 'both': 0.4}

Classifications:
{01100011 -> left, 10100111 -> right, 10100011 -> both}

Now that our training process is complete, let's see what the network thinks about an input that was not in the training data.

In [9]:
# This should evaluate to "left" since all `True` values are on the left side of the array.
lvr_net([True, True, True, False, False, False, False, False])

#### MNIST

Now let's test our system against something a bit more complicated. [MNIST](https://en.wikipedia.org/wiki/MNIST_database) is a dataset containing a variety of handwritten numbers in grayscale. To make each 28x28 image work with our network, we need to flatten the images into an array of strictly boolean values. We'll decide that any pixel value greater than `64` (out of `255`) should be considered `True`.

In [10]:
from qbn.datasets import get_mnist
(mnist_training, mnist_test) = get_mnist(true_threshold=64)

# Note the length of a single entry is 784 since each image is 28x28.
assert len(list(mnist_training.keys())[0]) == 28 * 28

Let's create the boolean network. I should mention here that we make the assumption that our network has periodic boundary conditions, so any input to a layer that is longer than the layer itself will simply wrap around to the first boolean function again. This means that although we can make our network arbitrarily small in either dimension, the output dimensionality will always be the same as the input dimensionality. In other words, we are always doing 784 calculations per layer regardless of the layer size. While this has many downsides, one upside is that it is far more likely that unique inputs will generate unique outputs, and so the chance of getting a "valid" categorization increases.

In [11]:
mnet = Network.build([392, 196, 98, 49, 24, 12, 6])

Like the previous example, let's start by checking the baseline accuracy of our network. You'll notice that each of the classification bitstrings are the size of an image itself.

In [12]:
mnet.accuracy(mnist_training)

Valid?:            True
Overall Accuracy:  0.01
Accuracy By Class: {'5': 0.01, '0': 0.01, '4': 0.01, '1': 0.02, '9': 0.01, '2': 0.01, '3': 0.01, '6': 0.01, '7': 0.01, '8': 0.01}

Classifications:
{1011111010011010111010001111101000101010111010001001101100101011101100011011111010101011111100001001101000101010111100011011111100111011100010011011101100101011101010011001101000101011100100111011101000101011101100011011101010101011111000111001111100111011101001111001101010111001111000001111101110101011101100001001101100111001101010001011101010101010111010111111101010101001101000001111111000111010111100011011101100101010111100011011101001111001101000001111111110101000111100111011111100011001111100011011111000101000111000001001111000111011101100001011101001111010111111111001101000101011101000111011101010101010100001111011101011111011101100111011101001111010111010011011101100011010111010001001111100111011101000001001111000101010111010111001111000101010111001101111101100111011 -> 5, 101111101001

Here's where we'll see firsthand the major downside of our required training algorithm. The process of training the network takes about a second per iteration. In my experience it is common to gain somewhere around 0.1-1% accuracy every hour of training time (with the gains becoming less and less over time).

In [ ]:
mnet.train_classical(mnist_training, changes_per_iter=10)

### Introduction to Neutral-Atom Quantum Computers

A [quantum computer](https://en.wikipedia.org/wiki/Quantum_computing) is a device that leverages quantum mechanical properties to perform computations. Such a computer works with qubits: the quantum variant of the classical binary bit. There are essentially two major technologies behind quantum computers at the moment: [Ion Traps](https://en.wikipedia.org/wiki/Trapped-ion_quantum_computer), and [Neutral-Atom Quantum Computers](https://en.wikipedia.org/wiki/Neutral_atom_quantum_computer). Ion Traps are a form of _Logical Quantum Computer_: they work by exploiting the quantum properties of their components to enable the computer to _abstractly_ perform computations without any major physical change in the device itself. Neutral-Atom Quantum Computers (NAQCs) are a form of _Analog Quantum Computer_: they work by building a physical "machine" composed of atoms that intract with each other naturally. This project leverages the [Aquila](https://arxiv.org/abs/2306.11727) NAQC developed by QuEra. Without making this notebook dozens of pages longer, I'll briefly describe how it works and what sorts of neat things we can do with it.

Aquila is a 256-qubit neutral atom quantum computer that exploits quantum states by physically arranging individual rubidium atoms in user-defined positions. Rubidium is positioned on the first column of the periodic table, meaning that its chemical and quantum mechanical properties share a similarity with hydrogen: mainly that it has a single electron in its outermost shell. When a pulse of light hits this electron, it has a chance to "puff" outward from the atom's nucleus into an "orbit" farther away. We call the initial state (before the pulse of light) the _ground state_, and the "puffed" state the _Rydberg state_. In quantum mechanical formalism, we use $\ket{g}$ to denote the _ground state_ and $\ket{r}$ to denote the _Rydberg state_.

![Rydberg States](docs/img/atom-simplified.png)

The _ground_ and _Rydberg_ states together make up the two possible _measurement states_ of a qubit (although reversed from what you might think, so $\ket{g}$ corresponds to $\ket{1}$). I explicitly say _measurement states_ because until a measurement on the system is made, each atom exists in a [superposition](https://en.wikipedia.org/wiki/Quantum_superposition) of both states. Mathematically this concept is represented like so (for a single atom):

\begin{equation}
\ket{\Psi} = c_0 \ket{g} + c_1 \ket{r}
\end{equation}

Where $\ket{\Psi}$ is the quantum (unmeasured) state of the system, $c_{0}$ is the _probability amplitude_ of measuring the state as $\ket{g}$, and $c_{1}$ is the _probability amplitude_ of measuring the state as $\ket{r}$. The "real" probability of measuring one state or another is obtained by squaring the corresponding $c$ values.

The last bit of quantum theory I need to talk about describes the _dynamics_ or _evolution_ of the quantum system. In Aquila, we perform a quantum computation by positioning a bunch of rubidium atoms on a 2D plane with some initial state using a series of lasers called "atomic tweezers". We then interact with the system with a _driving laser_ that causes certain atoms to enter or leave the Rydberg state. Atoms which are positioned close to each other such that their orbitals overlap during this transition will cause them to interact. The overall dynamics of this system, and indeed any quantum system, is determined by the time-dependent Schrödinger equation:

\begin{equation}
i \hbar \frac{\partial}{\partial t} \ket{\Psi} = \mathcal{H}(t) \ket{\Psi}
\end{equation}

Where $\hbar$ is the reduced Planck's constant, $i$ is the imaginary number, $t$ is time, $\ket{\Psi}$ is the quantum state of the system, and $\mathcal{H}(t)$ represents what's called the _Hamiltonian Operator_ which mutates the state of the system over time. For Aquila specifically, the Hamiltonian is defined by this ugly-looking equation:

\begin{equation}
\frac{\mathcal{H}(t)}{\hbar} = \sum_i \frac{\Omega(t)}{2} \left( e^{i \phi(t) } \ket{g_i} \bra{r_i} + e^{-i \phi(t) } \ket{r_i} \bra{g_i} \right) - \sum_i \Delta(t) \hat{n}_i + \sum_{i < j} V_{ij} \hat{n}_i \hat{n}_j.
\end{equation}

The only thing you need to know about the above is that the equation controlling the dynamics of the system contains three input parameters that can vary with time:

1. $\Omega(t)$ - The _Rabi Amplitude_, which determines the frequency at which each atom oscillates between the ground and
Rydberg states in the _absence_ of interactions with other atoms.
2. $\phi(t)$ - The _Phase_ of the driving lazer.
3. $\Delta(t)$ - The _Detuning_ of the driving lazer.

### Translating Boolean Functions to Neutral Atoms

So how does one take a boolean function and convert that into a computation with neutral atoms? Well, it turns out each boolean function _can_ be implemented via certain atom configurations. For instance, a traditional `NOT` gate can be implemented by two entangled atoms (either veritcally or horizontally). The top atom is the input qubit and the bottom atom is the output qubit.

In [13]:
from qbn.atoms import NOT_GATE
NOT_GATE

                                   Atom Positions                               
    ┌──────────────────────────────────────────────────────────────────────────┐
6.00┤                                     •                                    │
    │                                                                          │
    │                                                                          │
5.17┤                                                                          │
    │                                                                          │
    │                                                                          │
4.33┤                                                                          │
    │                                                                          │
    │                                                                          │
3.50┤                                                                          │
    │                       

Binary gates are much more complex. Below is what an `AND` gate looks like. Weirdly, the atom at the bottom represents the first input and the atom at the top right of the square represents the second input. The output of the gate is captured on the bottom left corner of the square.

In [14]:
from qbn.atoms import AND_GATE
AND_GATE

                                   Atom Positions                               
    ┌──────────────────────────────────────────────────────────────────────────┐
21.0┤•                                                                        •│
    │                                                                          │
    │                                                                          │
17.7┤                                                                          │
    │                                                                          │
    │                                                                          │
14.3┤                                                                          │
    │•                                                                        •│
    │                                                                          │
11.0┤                                                                          │
    │                       

To be able to build a quantum representation of our boolean network, we need to construct a lattice comprised of a variety of shapes similar to the above. Here's an example of a lattice representing a network comprised of only XNOR functions:

![XNOR Lattice](docs/img/xnor-lattice.png)

"Cool, so just build a lattice for each version of the network during training right?"

Unfortunately, the answer to the above question is "no". That approach would only replicate the classical training algorithm but compute the results of each run in a "quantum way". At the end of the day, we're still just randomly mutating a network until we find a better one. What we need to do is figure out how to think of some arrangement of atoms that effectively "relaxes" into the correct solution. Therefore, each "chunk" needs to be not the quantum version of a classical boolean function, but something else that can behave like _any arbitrary boolean function_. Ideally, this hypothetical configuration of atoms would have some kind of "internal memory" that switches it between the various equivalent boolean functions.

Enter the [Toffoli Gate](https://en.wikipedia.org/wiki/Toffoli_gate). 

### Toffoli Gates & Reversable Boolean Computations

At this point we're starting to reach truly novel stuff that pushes the limits of the hardware at our disposal. I'm going to be brief here (I promise this time). Toffoli gates require at minimum 3 qubits to construct, depending on whether chaining multiple Toffoli gates together is acceptable (not for us, unfortunately). Since each universal gate needs two input "qubit wires" and one output qubit wire, we're looking at 6 qubits per neuron of our network. This means we cannot, with current harware limitations, run a network with more than 42 neurons _in total_. In reality, you're probably looking at a limit of something like 10-21 neurons because each gate needs to be sufficiently isolated from the quantum effects of other gates. So you _might_ be able to get away with classifying the "left vs. right" problem for an array of length 5 or so. Hardly "Quantum Supremacy", am I right?

### Amazon Braket, Bloqade, & Grainger

So how does one just "run code" on an Aquila quantum computer? Before we even talk about what to run _on_ the machine, we have to first figure out how to gain access to one and how to interface with it. Believe it or not, anyone in the world can currently submit tasks to an Aquila quantum computer (and a variety of other ones as well) through an AWS resource called [Amazon Braket](https://aws.amazon.com/braket/). Amazon exposes a incredibly simple API called the [Amazon Braket SDK](https://github.com/amazon-braket/amazon-braket-sdk-python). Interaction via the SDK typically looks something like this:

In [ ]:
import boto3
import pandas as pd
from braket.ahs.atom_arrangement import AtomArrangement
from braket.aws import AwsDevice, AwsSession
from braket.devices import Devices

AWS_PROFILE = 'quintic-shock'

# Setup AWS Session.
session = AwsSession(boto_session=boto3.Session(profile_name=AWS_PROFILE))

# Register and Connect to an Aquila QC.
device = AwsDevice(Devices.QuEra.Aquila, aws_session=session)

# Display some technical information about the device's capabilities.
device_properties = device.properties.dict()
device_properties['paradigm']['lattice']['geometry']

In the above, we can see that the minimum allowed spacing between atoms is $4 \mu m$, with 256 "sites" (atoms) maximum. I won't go further into the contraints around atom placement, but it should be mentioned that there are indeed some limitations. I'm also not going to show more about the Amazon Braket SDK because to be completely frank, it's really difficult to work with. The _actual_ library we will be using is called [Bloqade](https://github.com/QuEraComputing/bloqade-python), which is written by QuEra as a wrapper around the Braket SDK.

With Bloqade, defining the positions of atoms is as simple as providing a list of xy coordinates (measured in $\mu m$):

In [15]:
from bloqade.atom_arrangement import ListOfLocations

example_atom_locations = ListOfLocations([
    (0.0, 0.0),
    (0.0, 5.0),
    (0.0, 9.0),
    (5.0, 2.0),
    (6.0, 7.0),
    (9.0, 10.0)
])

example_atom_locations

                                   Atom Positions                               
    ┌──────────────────────────────────────────────────────────────────────────┐
10.0┤                                                                         •│
    │                                                                          │
    │•                                                                         │
 8.3┤                                                                          │
    │                                                                          │
    │                                                 •                        │
 6.7┤                                                                          │
    │                                                                          │
    │                                                                          │
 5.0┤•                                                                         │
    │                       

Here's one I made in honor of KubeCon (I swear it's supposed to be the Kubernetes logo):

In [16]:
from qbn.atoms import KUBECON

KUBECON

                                   Atom Positions                               
    ┌──────────────────────────────────────────────────────────────────────────┐
10.2┤                                    •                                     │
    │                                                                          │
    │                                                                          │
 8.9┤                         •          •          •                          │
    │       •                                                         •        │
    │                                                        •                 │
 7.5┤                 •                   •                                    │
    │                         •                       •                        │
    │            •                   •    •     •                 •            │
 6.2┤                                     •                                    │
    │                      •

Honestly I'd add more here but I need to work on my powerpoint XD

## Solution

### Case 1: Left vs. Right

Let's start small by trying to optimize the left vs. right problem for a slightly larger array of values. Since it's actually really easy to compute these, we can actually generate all possible combinations, shuffle them, and then cut the set in half.

In [17]:
from qbn.datasets import get_left_vs_right
(lvr_training, lvr_test) = get_left_vs_right(length=16)

Now let's define our network and check its initial state. We'll also serialize the network to disk so that we can repeat our experiment with the same initial conditions.

In [18]:
from qbn.network import Network
lvrn = Network.build([16] * 8)
lvrn.layers

[4, 14, 9, 1, 14, 1, 4, 1, 6, 2, 8, 9, 7, 14, 4, 2]
[9, 7, 4, 9, 1, 2, 11, 11, 7, 7, 8, 9, 2, 9, 4, 6]
[8, 4, 1, 8, 4, 1, 2, 9, 13, 2, 8, 7, 13, 11, 7, 9]
[8, 9, 8, 11, 13, 11, 14, 9, 14, 11, 14, 14, 14, 14, 8, 11]
[8, 14, 4, 1, 7, 7, 8, 11, 8, 2, 13, 6, 14, 4, 4, 11]
[13, 6, 8, 14, 9, 6, 2, 4, 13, 14, 6, 1, 9, 2, 13, 9]
[7, 14, 13, 7, 1, 4, 2, 7, 4, 9, 1, 1, 14, 2, 4, 8]
[2, 8, 8, 11, 4, 6, 2, 4, 2, 11, 14, 4, 13, 1, 14, 2]

In [19]:
lvrn.accuracy(lvr_training)

Valid?:            False
Overall Accuracy:  0.0
Accuracy By Class: {'both': 0.0, 'left': 0.0, 'right': 0.0, 'neither': 0.0}

Classifications:
{0001000001110010 -> both}

In [ ]:
lvrn.save('lvrn.network')

Let's go through the process of training the network classically.

In [ ]:
lvrn.train_classical(lvr_training, max_iterations=100000)